# Table 2&3 (fixed effects regression)

In [64]:
import pandas as pd
import numpy as np
import matplotlib as plt

df = pd.read_stata('data/Authority.dta')
print(df.shape)
#list(df.columns)

(16127, 31)


* ***Construct Work Category Dummies***

In [65]:
df['OG03_dummy'] = 0
df.loc[(df['work_category']=='OG03')&(df['work_category']!=''),'OG03_dummy'] = 1

df['OG01_dummy'] = 0
df.loc[(df['work_category']=='OG01')&(df['work_category']!=''),'OG01_dummy'] = 1

df['OG_rest_dummy'] = 0
df.loc[(df['OG01_dummy']!=1)&(df['OG03_dummy']!=1)&(df['work_category']!=''),'OG_rest_dummy'] = 1

df['OG_dummy'] = 0
df.loc[df['work_category'].str[0:2] == 'OG','OG_dummy'] = 1

df['OS_dummy'] = 0
df.loc[df['work_category'].str[0:2] == 'OS','OS_dummy'] = 1

#print(df.shape) to check column number (16127,36)
#df.head()

* ***Treated vs Controls***

In [66]:
df['trend'] = df['year'] - 1999

df['trend_treat'] = df['trend']
df.loc[(df['authority_code']!=3090272)&(df['authority_code']!=3070001),'trend_treat'] = 0
#15225 real change made / no zeros in df['trend']
#print(df['trend_treat'].value_counts()) chekced

df['trend_control'] = df['trend']
df.loc[(df['authority_code']==3090272)|(df['authority_code']==3070001),'trend_control'] = 0
#902 real change made
#print(df['trend_control'].value_counts()) checked

* ***PA Specific***

In [67]:
df = df.sort_values(by='authority_code',ascending=True)
#auth = econ.group_id(df, cols = 'authority_code')
#print(auth) #dataframe

auth_list = df['authority_code'].values.tolist()
auth_list = list(set(auth_list))
#본격적으로 df에 id_auth 칼럼을 만들것임
#겹치는 부분 = authority_code야
#id_auth = group_id + 1 if df의 auth code == auth의 code

df['id_auth'] = 0
for i in range(len(df)):
    for j in range(len(auth_list)):
        if df.loc[i,'authority_code'] == auth_list[j]:
            df.loc[i,'id_auth'] = j+1

#for statement check            
#df.head
#print(max(df['id_path']))

* regression tools

In [68]:
#2001~2010 dummy
for j in range(2001,2011):
    df['_Iyear_'+str(j)] = 0
    for i in range(len(df)):
        if df.loc[i,'year'] == j:
            df.loc[i, '_Iyear_'+str(j)] = 1
            

In [69]:
#(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1) obs=1262 checked
df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
df_reg_exp.head()

,authority_code,year,auth_anno,fpsb_auction,n_bidders,discount,reserve_price,work_category,complex_work,delay_ratio,...,_Iyear_2001,_Iyear_2002,_Iyear_2003,_Iyear_2004,_Iyear_2005,_Iyear_2006,_Iyear_2007,_Iyear_2008,_Iyear_2009,_Iyear_2010
2546,3.0,2005.0,26.0,0.0,25.0,15.200000,1129754.0,OG02,3.0,40.547947,...,0,0,0,0,1,0,0,0,0,0
2544,3.0,2005.0,26.0,0.0,21.0,14.720000,1982000.0,OG01,2.0,17.362270,...,0,0,0,0,1,0,0,0,0,0
973,3.0,2002.0,23.0,0.0,72.0,17.879999,620000.0,OG03,1.0,NaN,...,0,1,0,0,0,0,0,0,0,0
2541,3.0,2005.0,26.0,0.0,136.0,21.830000,738612.0,OG03,1.0,0.000000,...,0,0,0,0,1,0,0,0,0,0
2549,3.0,2005.0,26.0,0.0,46.0,19.180000,435225.0,OS28,2.0,38.333332,...,0,0,0,0,1,0,0,0,0,0


# PanelOS fixed effect

In [70]:
# mutlicollinearity
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

In [71]:
#check multicollinearity
X = df_reg_exp.loc[:,['fpsb_auction','_Iyear_2001','_Iyear_2002','_Iyear_2003','_Iyear_2004','_Iyear_2005','_Iyear_2006','_Iyear_2007','_Iyear_2008','_Iyear_2009','_Iyear_2010']]
calc_vif(X)

#will be problematic if VIF >=10
#result: everything is under 10

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1687: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.uncentered_tss


,variables,VIF
0,fpsb_auction,1.153799
1,_Iyear_2001,1.000000
2,_Iyear_2002,1.000000
3,_Iyear_2003,1.028333
4,_Iyear_2004,1.032031
5,_Iyear_2005,1.083450
6,_Iyear_2006,1.009984
7,_Iyear_2007,NaN
8,_Iyear_2008,NaN
9,_Iyear_2009,NaN


In [72]:
!pip install linearmodels  
from linearmodels import PanelOLS
import statsmodels.api as sm

In [49]:
'''df_reg_exp = df_reg_exp.set_index(['authority_code', 'year'])
exog_vars = ['fpsb_auction','_Iyear_2001','_Iyear_2002','_Iyear_2003','_Iyear_2004','_Iyear_2005','_Iyear_2006']#,'_Iyear_2007','_Iyear_2008','_Iyear_2009','_Iyear_2010']     
mod = PanelOLS(df_reg_exp.discount, df_reg_exp[exog_vars], time_effects=True)  
fe_res = mod.fit()  
print(fe_res)  
#y variable syntax error'''

AbsorbingEffectError: 
The model cannot be estimated. The included effects have fully absorbed
one or more of the variables. This occurs when one or more of the dependent
variable is perfectly explained using the effects included in the model.

The following variables or variable combinations have been fully absorbed
or have become perfectly collinear after effects are removed:

          _Iyear_2001
          _Iyear_2002
          _Iyear_2003
          _Iyear_2004
          _Iyear_2005
          _Iyear_2006

Set drop_absorbed=True to automatically drop absorbed variables.


# other commands

In [88]:
# 1. statasmodel regression -> coeff vals/ std error dont match 
# singular matrix check; [_Iyear_2007','_Iyear_2008','_Iyear_2009','_Iyear_2010']
#(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1) obs=1262 checked
df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
df_reg_exp.head()

x = df_reg_exp[['fpsb_auction','_Iyear_2001','_Iyear_2002','_Iyear_2003','_Iyear_2004','_Iyear_2005','_Iyear_2006','_Iyear_2007','_Iyear_2008','_Iyear_2009','_Iyear_2010','authority_code']]
y = df_reg_exp['discount']
x = sm.add_constant(x)
model = sm.OLS(y,x).fit(cov_type='cluster',cov_kwds = {'groups': df_reg_exp['year']})
result = model.predict(x)
model.summary()

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 12, but rank is 2
  'rank is %d' % (J, J_), ValueWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               discount   R-squared:                       0.456
Model:                            OLS   Adj. R-squared:                  0.452
Method:                 Least Squares   F-statistic:                     384.0
Date:                Mon, 29 Jun 2020   Prob (F-statistic):           4.66e-07
Time:                        17:53:07   Log-Likelihood:                -4013.6
No. Observations:                1262   AIC:                             8045.
Df Residuals:                    1253   BIC:                             8091.
Df Model:                           8                                         
Covariance Type:              cluster                                         
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const              9.8300      0.587     16.752      0.000       8.680      10.980
fpsb_auction      12.5675      2.007      6.262      0.000       8.634      16.501
_Iyear_2001        1.9164      0.247      7.763      0.000       1.433       2.400
_Iyear_2002        1.0890      0.354      3.078      0.002       0.395       1.783
_Iyear_2003        0.6293      0.598      1.052      0.293      -0.543       1.801
_Iyear_2004        2.3247      0.597      3.896      0.000       1.155       3.494
_Iyear_2005        4.1639      0.616      6.764      0.000       2.957       5.370
_Iyear_2006        4.2759      0.592      7.221      0.000       3.115       5.436
_Iyear_2007             0          0        nan        nan           0           0
_Iyear_2008             0          0        nan        nan           0           0
_Iyear_2009             0          0        nan        nan           0           0
_Iyear_2010             0          0        nan        nan           0           0
authority_code  2.012e-06   5.52e-07      3.643      0.000     9.3e-07     3.1e-06
==============================================================================
Omnibus:                      112.188   Durbin-Watson:                   1.648
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              221.763
Skew:                           0.572   Prob(JB):                     7.00e-49
Kurtosis:                       4.705   Cond. No.                          inf
==============================================================================

Warnings:
[1] Standard Errors are robust to cluster correlation (cluster)
[2] The smallest eigenvalue is      0. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [51]:
#2 econtools
!pip install econtools
import econtools as econ
import econtools.metrics as mt

In [53]:
df_reg_exp.head()

,authority_code,year,auth_anno,fpsb_auction,n_bidders,discount,reserve_price,work_category,complex_work,delay_ratio,...,_Iyear_2001,_Iyear_2002,_Iyear_2003,_Iyear_2004,_Iyear_2005,_Iyear_2006,_Iyear_2007,_Iyear_2008,_Iyear_2009,_Iyear_2010
2546,3.0,2005.0,26.0,0.0,25.0,15.200000,1129754.0,OG02,3.0,40.547947,...,0,0,0,0,1,0,0,0,0,0
2544,3.0,2005.0,26.0,0.0,21.0,14.720000,1982000.0,OG01,2.0,17.362270,...,0,0,0,0,1,0,0,0,0,0
973,3.0,2002.0,23.0,0.0,72.0,17.879999,620000.0,OG03,1.0,NaN,...,0,1,0,0,0,0,0,0,0,0
2541,3.0,2005.0,26.0,0.0,136.0,21.830000,738612.0,OG03,1.0,0.000000,...,0,0,0,0,1,0,0,0,0,0
2549,3.0,2005.0,26.0,0.0,46.0,19.180000,435225.0,OS28,2.0,38.333332,...,0,0,0,0,1,0,0,0,0,0


In [112]:
#Iwork 1~42 pattern unfound
#G인지 S인지 판단
#G이면 sorted된 거에서 인덱스 +1 =Iwork_nm
#S이면 sorted된 거에서 index + 17

work_ca = df['work_category'].values.tolist()
work_ca = list(set(work_ca))
#print(work_ca)
#total 42

os_list = []
og_list = []
for i in range(len(work_ca)):
    if work_ca[i][1]=='G':
        work_ca[i] = int(work_ca[i][2:])
        og_list.append(work_ca[i])
    else:
        work_ca[i] = int(work_ca[i][2:])
        os_list.append(work_ca[i])

os_list = sorted(os_list)
og_list = sorted(og_list)


print(df.loc[0,'work_category'][1])

S


In [176]:
work_ca_order = og_list + os_list
#work_ca_order 대신 (1,43)

for i in range(1,43):
    name = 'Iwork_ca_'+str(i+1) 
    df[name] = 0
    for j in range(len(df)):
        if df.loc[j,'work_category'][1] == 'G':
            work_OG = int(df.loc[j, 'work_category'][2:])
            for g in range(len(og_list)):
                if work_OG == og_list[g]:
                    idx_g = str(g+1)
                    df.loc[j, 'Iwork_ca_' + idx_g] = 1
        else:
            work_OS = int(df.loc[j, 'work_category'][2:])
            for s in range(len(os_list)):
                if work_OS == os_list[s]:
                    idx_s = str(s + 17)
                    df.loc[j, 'Iwork_ca_'+ idx_s] = 1

KeyboardInterrupt: 

In [175]:
df.head()

,authority_code,year,auth_anno,fpsb_auction,n_bidders,discount,reserve_price,work_category,complex_work,delay_ratio,...,Iwork_ca_25,Iwork_ca_27,Iwork_ca_15,Iwork_ca_42,Iwork_ca_14,Iwork_ca_21,Iwork_ca_30,Iwork_ca_16,Iwork_ca_5,Iwork_ca_43
15628,2.0,2006.0,17.0,0.0,46.0,13.730000,154926.0000,OG03,1.0,NaN,...,0,0,0,0,0,0,0,0,0,0
1830,2.0,2003.0,15.0,0.0,30.0,17.209999,252500.0000,OS28,2.0,10.0,...,0,0,0,0,0,0,0,0,0,0
14964,2.0,2008.0,19.0,0.0,68.0,21.049999,281850.9375,OG03,1.0,NaN,...,0,0,0,0,0,0,0,0,0,0
1731,2.0,2003.0,15.0,0.0,25.0,14.400000,387800.0000,OG01,2.0,0.0,...,0,0,0,0,0,0,0,0,0,0
15544,2.0,2008.0,19.0,0.0,23.0,22.799999,623500.0000,OG03,1.0,NaN,...,0,0,0,0,0,0,0,0,0,0


In [169]:
#PanelA_2
reg_col = ['fpsb_auction','_Iyear_2001','_Iyear_2002','_Iyear_2003','_Iyear_2004','_Iyear_2005','_Iyear_2006','_Iyear_2007','_Iyear_2008','_Iyear_2009','_Iyear_2010']
for i in range(len(work_ca_order)):
    reg_col.append('Iwork_ca_'+str(i+1))

reg_col = reg_col + ['reserve_price', 'municipality']
print(reg_col)

df_reg_exp2 = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]

x = df_reg_exp2[reg_col]
y = df_reg_exp2['discount']
model = sm.OLS(y,x).fit()
predictions = model.predict(x)
model.summary()


['fpsb_auction', '_Iyear_2001', '_Iyear_2002', '_Iyear_2003', '_Iyear_2004', '_Iyear_2005', '_Iyear_2006', '_Iyear_2007', '_Iyear_2008', '_Iyear_2009', '_Iyear_2010', 'Iwork_ca_1', 'Iwork_ca_2', 'Iwork_ca_3', 'Iwork_ca_4', 'Iwork_ca_5', 'Iwork_ca_6', 'Iwork_ca_7', 'Iwork_ca_8', 'Iwork_ca_9', 'Iwork_ca_10', 'Iwork_ca_11', 'Iwork_ca_12', 'Iwork_ca_13', 'Iwork_ca_14', 'Iwork_ca_15', 'Iwork_ca_16', 'Iwork_ca_17', 'Iwork_ca_18', 'Iwork_ca_19', 'Iwork_ca_20', 'Iwork_ca_21', 'Iwork_ca_22', 'Iwork_ca_23', 'Iwork_ca_24', 'Iwork_ca_25', 'Iwork_ca_26', 'Iwork_ca_27', 'Iwork_ca_28', 'Iwork_ca_29', 'Iwork_ca_30', 'Iwork_ca_31', 'Iwork_ca_32', 'Iwork_ca_33', 'Iwork_ca_34', 'Iwork_ca_35', 'Iwork_ca_36', 'Iwork_ca_37', 'Iwork_ca_38', 'Iwork_ca_39', 'Iwork_ca_40', 'Iwork_ca_41', 'Iwork_ca_42', 'reserve_price', 'municipality']


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               discount   R-squared:                       0.552
Model:                            OLS   Adj. R-squared:                  0.538
Method:                 Least Squares   F-statistic:                     39.66
Date:                Sun, 28 Jun 2020   Prob (F-statistic):          3.91e-184
Time:                        19:51:03   Log-Likelihood:                -3890.5
No. Observations:                1262   AIC:                             7859.
Df Residuals:                    1223   BIC:                             8060.
Df Model:                          38                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
fpsb_auction     17.5108      0.657     26.664      0.000      16.222      18.799
_Iyear_2001       0.7293      0.633      1.151      0.250      -0.513       1.972
_Iyear_2002      -0.9324      0.606     -1.539      0.124      -2.121       0.256
_Iyear_2003      -1.7563      0.634     -2.771      0.006      -3.000      -0.513
_Iyear_2004      -0.2013      0.607     -0.332      0.740      -1.392       0.990
_Iyear_2005       1.3545      0.621      2.181      0.029       0.136       2.573
_Iyear_2006       1.9738      0.684      2.886      0.004       0.632       3.316
_Iyear_2007    1.147e-08   1.54e-09      7.428      0.000    8.44e-09    1.45e-08
_Iyear_2008   -1.639e-09   4.37e-09     -0.375      0.708   -1.02e-08    6.94e-09
_Iyear_2009   -3.749e-09   4.42e-09     -0.848      0.397   -1.24e-08    4.93e-09
_Iyear_2010    3.049e-08   8.74e-09      3.489      0.001    1.33e-08    4.76e-08
Iwork_ca_1       10.3571      0.628     16.481      0.000       9.124      11.590
Iwork_ca_2       10.2521      0.878     11.681      0.000       8.530      11.974
Iwork_ca_3        9.8011      0.561     17.478      0.000       8.701      10.901
Iwork_ca_4        5.4449      3.842      1.417      0.157      -2.092      12.982
Iwork_ca_5     5.907e-09   1.36e-09      4.333      0.000    3.23e-09    8.58e-09
Iwork_ca_6       10.6154      0.915     11.601      0.000       8.820      12.411
Iwork_ca_7       16.2351      3.169      5.123      0.000      10.017      22.453
Iwork_ca_8       12.1131      1.152     10.511      0.000       9.852      14.374
Iwork_ca_9     5.532e-15   1.65e-15      3.352      0.001    2.29e-15    8.77e-15
Iwork_ca_10      20.0210      3.155      6.346      0.000      13.831      26.211
Iwork_ca_11      14.5237      1.235     11.764      0.000      12.102      16.946
Iwork_ca_12      18.9063      2.280      8.292      0.000      14.433      23.380
Iwork_ca_13   -4.392e-16   1.82e-15     -0.241      0.810   -4.02e-15    3.14e-15
Iwork_ca_14   -3.261e-15   2.53e-15     -1.291      0.197   -8.22e-15     1.7e-15
Iwork_ca_15   -8.561e-15    3.7e-15     -2.311      0.021   -1.58e-14   -1.29e-15
Iwork_ca_16    2.315e-15   2.03e-15      1.142      0.254   -1.66e-15    6.29e-15
Iwork_ca_17       5.8929      3.839      1.535      0.125      -1.638      13.424
Iwork_ca_18      17.1901      2.483      6.922      0.000      12.318      22.062
Iwork_ca_19    3.705e-15   9.39e-16      3.945      0.000    1.86e-15    5.55e-15
Iwork_ca_20    1.846e-15   2.16e-15      0.855      0.392   -2.39e-15    6.08e-15
Iwork_ca_21      16.5846      5.379      3.084      0.002       6.033      27.137
Iwork_ca_22      14.6686      1.084     13.526      0.000      12.541      16.796
Iwork_ca_23      13.9334      5.400      2.580      0.010       3.338      24.529
Iwork_ca_24      15.4402      2.435      6.341   

In [170]:
del reg_col[reg_col.index('_Iyear_2007')]
del reg_col[reg_col.index('_Iyear_2008')]
del reg_col[reg_col.index('_Iyear_2009')]
del reg_col[reg_col.index('_Iyear_2010')]

del reg_col[reg_col.index('Iwork_ca_5')]
del reg_col[reg_col.index('Iwork_ca_9')]
del reg_col[reg_col.index('Iwork_ca_13')]
del reg_col[reg_col.index('Iwork_ca_14')]
del reg_col[reg_col.index('Iwork_ca_15')]
del reg_col[reg_col.index('Iwork_ca_16')]
del reg_col[reg_col.index('Iwork_ca_19')]
del reg_col[reg_col.index('Iwork_ca_20')]
del reg_col[reg_col.index('Iwork_ca_25')]
del reg_col[reg_col.index('Iwork_ca_29')]
del reg_col[reg_col.index('Iwork_ca_40')]
del reg_col[reg_col.index('Iwork_ca_41')]


print(reg_col)


#mt.reg(df_reg_exp2,'discount',reg_col, fe_name = 'authority_code', cluster = 'auth_anno')

['fpsb_auction', '_Iyear_2001', '_Iyear_2002', '_Iyear_2003', '_Iyear_2004', '_Iyear_2005', '_Iyear_2006', 'Iwork_ca_1', 'Iwork_ca_2', 'Iwork_ca_3', 'Iwork_ca_4', 'Iwork_ca_6', 'Iwork_ca_7', 'Iwork_ca_8', 'Iwork_ca_10', 'Iwork_ca_11', 'Iwork_ca_12', 'Iwork_ca_17', 'Iwork_ca_18', 'Iwork_ca_21', 'Iwork_ca_22', 'Iwork_ca_23', 'Iwork_ca_24', 'Iwork_ca_26', 'Iwork_ca_27', 'Iwork_ca_28', 'Iwork_ca_30', 'Iwork_ca_31', 'Iwork_ca_32', 'Iwork_ca_33', 'Iwork_ca_34', 'Iwork_ca_35', 'Iwork_ca_36', 'Iwork_ca_37', 'Iwork_ca_38', 'Iwork_ca_39', 'Iwork_ca_42', 'reserve_price', 'municipality']


LinAlgError: Singular matrix

In [ ]:
#pop version
#df_reg_pop = df[(df['turin_pr_sample']==1)&(df['ctrl_pop_turin_pr_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]